In [ ]:
%run config

### API Query to get more specific details for each activity, need to pass each activity off individually 

In [ ]:
full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [ ]:
#grab all of the disinct activity IDs 
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
#test before API call, so we'll see what is in storage first.
#see what happens if we don't have a datset to write 
segmget_historical_dataset(

In [ ]:
subset = full_activity_ids[:30]

In [ ]:
def query_segments(activity_ids):
    import pandas as pd
        #can get the invdividal activites
    dataframe = pd.DataFrame()
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        activity_id_list =[]
        segment_id_list =[]
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)

            count += 1 
        
        columns = ['segment_id', 'activity_id']
        extracted_data = [segment_id_list, activity_id_list]
        segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

        dataframe.append(segment_df)

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(dataframe)

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("activity_information")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df







In [ ]:

def query_segments_test(activity_ids):
    import pandas as pd
        #can get the invdividal activites
    dataframe = pd.DataFrame()
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        activity_id_list =[]
        segment_id_list =[]
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                   
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))
            
            df = dataframe.append(segment_df)
            count += 1 

    #convert pandas df to spark


    return df


In [ ]:
subset = full_activity_ids[70:76]
pleasework=  query_segments_test(subset)

In [ ]:
subset

In [ ]:
testing_dataframe = pd.DataFrame()
activity_list = []
subset = full_activity_ids[25:30]
count = 0
import pandas as pd
for id in subset:
    activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': 1}
    my_activity = requests.get(activity_id_urls, headers=header, params=param).json()
    activity_id = my_activity['segment_efforts'][count]['activity']['id']
    column = ["pleasework"]
    extracted_data = [activity_id]
    dict_to_pass = dict(zip(column, extracted_data))
    pdf = pd.DataFrame.from_dict(data = dict_to_pass, orient= 'index', columns= column)
    testing_dataframe.append(pdf)
    count += 1





In [ ]:
count = 0
for i in range(len([1,2,4])):
    print(count)
    count += 1

In [ ]:
pdf = pd.DataFrame.from_dict(dict_to_pass, orient='index', columns = column)

In [ ]:
pdf

In [ ]:
my_activity

In [ ]:
for i in subset:
    print(i)

In [ ]:
pleasework

In [ ]:
pleasework['activity_id'].unique()

In [ ]:
from distutils.log import error
segment_effort_path

try:
    historical_dataframe = spark.read.format("delta").load(segment_effort_path)
except:
    error

if (historical_dataframe.count() > 0):
    return historical_dataframe
else:
    


return dataset 

In [ ]:
import pandas as pd
try:
    #try to read from storage
    pd.read_csv('file.csv')
except:
    #print("No file") 
    try: 
        pd.read_csv('file.csv')
    except:
        print( "No File x2")
finally:
    print("1+1")


In [ ]:
import pandas as pd
try:
    #try to read from storage
    pd.read_csv('file.csv')
except: 
    try:
    #if that fails, write the current dataframe to storage
        write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )
    except:
        print("No Dataframe")
    finally:
        historical_dataframe = spark.read.format("delta").load(storagepath)

In [ ]:
#run activites through api call

#

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 